In [1]:
import io
import fitz
import re
from PIL import Image, ImageDraw, ImageFont, ImageColor
import operator
import os
import pandas as pd 
import numpy as np
from datetime import datetime
import math
from tqdm import tqdm
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont

In [2]:
"""INDEX 1"""
pdf_dir = "../input/NOUVELLE FLORE DU LIBAN ET DE LA SYRIE 1.pdf"
doc = fitz.open(pdf_dir)
pages = [doc[i] for i in range(doc.page_count)]
"""INDEX 1"""

TARGET_DPI = 300
mat = fitz.Matrix(TARGET_DPI/ 72, TARGET_DPI/ 72)

indent_groups = []
indent_err = 15

In [3]:
def flags_decomposer(flags):
    """Make font flags human readable."""
    l = []
    if flags & 2 ** 0:
        l.append("superscript")
    if flags & 2 ** 1:
        l.append("italic")
    if flags & 2 ** 2:
        l.append("serifed")
    else:
        l.append("sans")
    if flags & 2 ** 3:
        l.append("monospaced")
    else:
        l.append("proportional")
    if flags & 2 ** 4:
        l.append("bold")
    return l

In [4]:
def is_author(name):
    return False

def is_epithet(span): #just checks if is bold
    if 'bold' in  flags_decomposer(span['flags']): #binomial species / sub sp description
        return True

def is_genus(word):
    pruned_word = ''.join(c for c in word if c.isalpha())
    return len(pruned_word) >= 3 and pruned_word.isupper()

def is_family(word):
    return is_genus(word)

def is_center_aligned(coords):
    return False

def is_paragraph_start(coords):
    return True

def is_separate(coords): #vertical / spatial separation of boxes 
    return True

def is_new_entry(spans, coords, num_words_limit = 6, alignment = None):
    words_seen = 0 
    start_flag = False

    if not is_paragraph_start(coords) or not is_separate(coords):
        return False

    if is_center_aligned(coords): 
        return True

    while words_seen < num_words_limit: 
        for s in spans:
            s_text_list = s['text'].split(" ")[:6-words_seen] #making sure we are only working with the first 6 words in this function
            words_seen += len(s_text_list)
            
            print(s)
            if is_epithet(s):
                return True
            else:
                for w in s_text_list: 
                    if (is_genus(w) or is_family(w)) and not is_author(w):
                        return True
                
    return False

